In [7]:
from rev_ai import apiclient
import os

ACCESS_TOKEN = "022noQShzOFkDaBPRnGKcWRfLg_Caq76eOW5FodAT_nyILtUKdPZAJ7_G97GXvmUGTPTpDnyTXIgU0SBZXngPXpA94pW4"
AUDIO_FILE = "New Recording 49.m4a"

client = apiclient.RevAiAPIClient(ACCESS_TOKEN)

# 1. Submit file for transcription
job = client.submit_job_local_file(AUDIO_FILE)

print(f"Submitted job: {job.id}")

# 2. Wait until done
job_details = client.get_job_details(job.id)
while job_details.status not in ["transcribed", "failed"]:
    import time
    print("Waiting for transcription...")
    time.sleep(5)
    job_details = client.get_job_details(job.id)

# 3. Get transcript object
transcript = client.get_transcript_object(job.id)
output_text = ""
last_end = 0.0

for monologue in transcript.monologues:
    for element in monologue.elements:
        # Some elements may not have timestamps (like punctuation)
        if hasattr(element, "ts") and hasattr(element, "end_ts"):
            start = element.ts
            if start and last_end and (start - last_end) >= 3.0:
                pause_len = round(start - last_end)
                output_text += f" [PAUSE:{pause_len}s] "
            output_text += element.value + " "
            last_end = element.end_ts
        else:
            # punctuation or other tokens without timestamps
            output_text += element.value

print(output_text.strip())


Submitted job: DY4q9p5tzt4od9tr
Waiting for transcription...
Waiting for transcription...
Waiting for transcription...
<silence> Hello, my name is Ani and uh, as you know, I'm trying to build a project. You know, I'm facing very much difficulties right now because, uh, in my mind there is a lot of things happening right now. Uh, but I think so I can welcome that because God is in my side. I know I'm just making this audio to try and test my <inaudible> that is it able to detect any pauses between the audios and show it, show it in the, you know, text. Uh, that's why I don't use this words like, uh, but I'm using these words so that my model can become better. Thank you.


In [5]:
from rev_ai import apiclient
from pydub import AudioSegment, silence
import time

# ---- 1. Rev AI transcription ----
ACCESS_TOKEN = "022noQShzOFkDaBPRnGKcWRfLg_Caq76eOW5FodAT_nyILtUKdPZAJ7_G97GXvmUGTPTpDnyTXIgU0SBZXngPXpA94pW4"   # put your Rev AI token here
AUDIO_FILE = "New Recording 48.m4a"   # your M4A file

client = apiclient.RevAiAPIClient(ACCESS_TOKEN)

job = client.submit_job_local_file(AUDIO_FILE)
print(f"Submitted job: {job.id}")

# wait until transcription completes
job_details = client.get_job_details(job.id)
while job_details.status not in ["transcribed", "failed"]:
    print("Waiting for transcription...")
    time.sleep(5)
    job_details = client.get_job_details(job.id)

if job_details.status == "failed":
    raise RuntimeError("Rev AI transcription failed!")

transcript = client.get_transcript_object(job.id)

# ---- 2. Detect pauses with PyDub ----
# load M4A file (requires ffmpeg installed)
audio = AudioSegment.from_file(AUDIO_FILE, format="m4a")

silences = silence.detect_silence(
    audio,
    min_silence_len=3000,  # 3 sec
    silence_thresh=-40     # adjust if needed (depends on recording volume)
)
# silence list = [(start_ms, end_ms), ...]

# ---- 3. Merge transcript + pauses ----
output_text = ""
last_end = None

for monologue in transcript.monologues:
    for element in monologue.elements:
         print("Element:", element)
         if hasattr(element, "ts") and hasattr(element, "end_ts"):  
            start = element.ts
            if last_end is not None:
                pause = start - last_end
                if pause >= 3.0:  # pause threshold in seconds
                    output_text += f"[PAUSE:{int(round(pause))}s] "
            
            # Add the actual word/punctuation
            if element.value in [".", ",", "?", "!", ";", ":"]:
                output_text = output_text.rstrip() + element.value + " "
            else:
                output_text += element.value + " "
            
            last_end = element.end_ts
         else:
            # if no timestamps, just add the value
            output_text += element.value + " "

# Cleanup spacing
output_text = " ".join(output_text.split())


Submitted job: Cfp5iXrhTHLNOooM
Waiting for transcription...
Waiting for transcription...
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE115400>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FDFC2D50>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FDFC3750>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE023BB0>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE023230>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE08AF90>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE08DBF0>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE08E030>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE06B550>
Element: <rev_ai.models.asynchronous.transcript.Element object at 0x00000190FE06B750>
Element: <rev_ai.models.asynchronous.transcript.El

In [9]:
from rev_ai import apiclient
from pydub import AudioSegment, silence
import time

# ---- 1. Rev AI transcription ----
ACCESS_TOKEN = "022noQShzOFkDaBPRnGKcWRfLg_Caq76eOW5FodAT_nyILtUKdPZAJ7_G97GXvmUGTPTpDnyTXIgU0SBZXngPXpA94pW4"
AUDIO_FILE = "New Recording 48.m4a"

client = apiclient.RevAiAPIClient(ACCESS_TOKEN)

job = client.submit_job_local_file(
        AUDIO_FILE,
        verbatim=True,  # This should include filler words AND timestamps
        remove_disfluencies=False
    )
    
# Wait until transcription completes
job_details = client.get_job_details(job.id)
while job_details.status not in ["transcribed", "failed"]:
    print("Waiting for transcription...")
    time.sleep(5)
    job_details = client.get_job_details(job.id)

if job_details.status == "failed":
    raise RuntimeError("Rev AI transcription failed!")

transcript = client.get_transcript_object(job.id)

# ---- 2. Build transcript with pause detection ----
output_text = ""
last_end_time = None

for monologue in transcript.monologues:
    for element in monologue.elements:
        # Debug: Print what we actually have access to
        print(f"Element value: '{element.value}'")
        print(f"Element start time: {getattr(element, 'ts', 'No timestamp')}")
        print(f"Element end time: {getattr(element, 'end_ts', 'No timestamp')}")
        print("---")
        
        # Check if element has timestamps
        if hasattr(element, 'ts') and element.ts is not None:
            start_time = element.ts
            
            # Check for pause since last element
            if last_end_time is not None:
                pause_duration = start_time - last_end_time
                if pause_duration >= 3.0:  # 3 second threshold
                    output_text += f" [PAUSE:{pause_duration:.1f}s] "
            
            # Update last end time
            if hasattr(element, 'end_ts') and element.end_ts is not None:
                last_end_time = element.end_ts
            else:
                last_end_time = start_time  # fallback to start time
        
        # Add the element value (word/punctuation)
        if element.value and element.value.strip():  # Only add non-empty values
            if element.value in [".", ",", "?", "!", ";", ":"]:
                # Remove space before punctuation
                output_text = output_text.rstrip() + element.value + " "
            else:
                output_text += element.value + " "

# Clean up extra spaces
output_text = " ".join(output_text.split())

print("\n" + "="*50)
print("FINAL TRANSCRIPT WITH PAUSES:")
print("="*50)
print(output_text)
print("="*50)


Waiting for transcription...
Waiting for transcription...
Element value: 'Hello'
Element start time: No timestamp
Element end time: No timestamp
---
Element value: ','
Element start time: No timestamp
Element end time: No timestamp
---
Element value: ' '
Element start time: No timestamp
Element end time: No timestamp
---
Element value: 'my'
Element start time: No timestamp
Element end time: No timestamp
---
Element value: ' '
Element start time: No timestamp
Element end time: No timestamp
---
Element value: 'name'
Element start time: No timestamp
Element end time: No timestamp
---
Element value: ' '
Element start time: No timestamp
Element end time: No timestamp
---
Element value: 'is'
Element start time: No timestamp
Element end time: No timestamp
---
Element value: ' '
Element start time: No timestamp
Element end time: No timestamp
---
Element value: 'Brenda'
Element start time: No timestamp
Element end time: No timestamp
---
Element value: ' '
Element start time: No timestamp
Elemen

Error: 401 Client Error: Unauthorized for url: https://api.rev.ai/speechtotext/v1/jobs; Server Response : {"title":"Authorization has been denied for this request.","status":401}


## Crisper


In [4]:
from pydub import AudioSegment

# Convert M4A to WAV
audio = AudioSegment.from_file("WhatsApp Audio 2025-09-02 at 7.12.44 PM.mp4", format="mp4")
audio.export("WhatsApp Audio 2025-09-02 at 7.12.44 PM.wav", format="wav")

print("Converted M4A to WAV successfully!")


Converted M4A to WAV successfully!


In [12]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

def transcribe_with_crisper_whisper(audio_file, pause_threshold=3.0):
    """
    Use CrisperWhisper for verbatim transcription with filler words and timestamps
    """
    # Setup
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    
    model_id = "nyrahealth/CrisperWhisper"
    
    print("Loading CrisperWhisper model...")
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, 
        torch_dtype=torch_dtype, 
        low_cpu_mem_usage=True, 
        use_safetensors=True
    )
    model.to(device)
    
    processor = AutoProcessor.from_pretrained(model_id)
    
    # Create pipeline with word-level timestamps
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps='word',  # This ensures word-level timestamps
        torch_dtype=torch_dtype,
        device=device,
    )
    
    print(f"Transcribing: {audio_file}")
    result = pipe(audio_file)
    
    # Process chunks to add pause markers
    chunks = result.get("chunks", [])
    final_transcript = ""
    
    for i, chunk in enumerate(chunks):
        final_transcript += chunk["text"]
        
        # Check for pauses between chunks
        if i < len(chunks) - 1:
            current_end = chunk.get("timestamp", [None, None])[1]
            next_start = chunks[i + 1].get("timestamp", [None, None])[0]
            
            if current_end and next_start:
                pause_duration = next_start - current_end
                if pause_duration >= pause_threshold:
                    final_transcript += f" [PAUSE:{pause_duration:.1f}s]"
    
    return {
        "transcript": final_transcript,
        "chunks_with_timestamps": chunks,
        "raw_result": result
    }

# Usage
try:
    # You need to install: pip install transformers torch datasets accelerate
    # And login: huggingface-cli login
    
    result = transcribe_with_crisper_whisper("New Recording 48.wav", pause_threshold=3.0)
    
    print("\n" + "="*60)
    print("CRISPERWHISPER TRANSCRIPT WITH FILLER WORDS AND PAUSES:")
    print("="*60)
    print(result["transcript"])
    print("="*60)
    
    # Print word-level timestamps
    print("\nWORD-LEVEL TIMESTAMPS:")
    print("-" * 30)
    for chunk in result["chunks_with_timestamps"]:
        if chunk.get("timestamp"):
            start, end = chunk["timestamp"]
            print(f"{chunk['text']} | {start:.2f}s - {end:.2f}s")
    
except Exception as e:
    print(f"Error: {e}")
    print("Make sure you have installed the required packages and logged into Hugging Face")


Loading CrisperWhisper model...


Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


Transcribing: New Recording 48.wav


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


Error: The size of tensor a (2) must match the size of tensor b (0) at non-singleton dimension 1
Make sure you have installed the required packages and logged into Hugging Face


In [ ]:
from faster_whisper import WhisperModel
import torch

def transcribe_with_faster_crisper(audio_file, pause_threshold=3.0):
    """
    Use faster-whisper version of CrisperWhisper - most stable option
    """
    model_name = 'nyrahealth/faster_CrisperWhisper'
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    compute_type = "float16" if torch.cuda.is_available() else "float32"
    
    print(f"Loading Faster CrisperWhisper on {device}...")
    model = WhisperModel(model_name, device=device, compute_type=compute_type)
    
    print(f"Transcribing: {audio_file}")
    segments, info = model.transcribe(
        audio_file, 
        beam_size=1, 
        language='en', 
        word_timestamps=True,
        without_timestamps=False,
        vad_filter=True,  # Voice activity detection
        vad_parameters=dict(min_silence_duration_ms=500)
    )
    
    # Process segments to build transcript with pauses
    transcript = ""
    word_timestamps = []
    last_end_time = None
    
    for segment in segments:
        # Check for pause between segments
        if last_end_time is not None:
            pause_duration = segment.start - last_end_time
            if pause_duration >= pause_threshold:
                transcript += f" [PAUSE:{pause_duration:.1f}s] "
        
        # Add segment text (includes filler words)
        transcript += segment.text
        last_end_time = segment.end
        
        # Collect word-level timestamps if available
        if hasattr(segment, 'words') and segment.words:
            for word in segment.words:
                word_timestamps.append({
                    "word": word.word,
                    "start": word.start,
                    "end": word.end
                })
        
        # Print segment for debugging
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
    
    return {
        "transcript": transcript.strip(),
        "word_timestamps": word_timestamps,
        "segments": list(segments)  # Convert generator to list
    }

# Install faster-whisper first
# pip install faster-whisper

try:
    result = transcribe_with_faster_crisper("New Recording 48.wav", pause_threshold=3.0)
    
    print("\n" + "="*60)
    print("FASTER CRISPERWHISPER TRANSCRIPT WITH FILLER WORDS AND PAUSES:")
    print("="*60)
    print(result["transcript"])
    print("="*60)
    
    if result["word_timestamps"]:
        print("\nWORD-LEVEL TIMESTAMPS:")
        print("-" * 40)
        for word_info in result["word_timestamps"][:20]:  # Show first 20 words
            print(f"'{word_info['word']}' | {word_info['start']:.2f}s - {word_info['end']:.2f}s")
    
except Exception as e:
    print(f"Error with Faster CrisperWhisper: {e}")


c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python313\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Loading Faster CrisperWhisper on cuda...
Error with Faster CrisperWhisper: CUDA failed with error out of memory


In [17]:
from faster_whisper import WhisperModel
import torch

def transcribe_with_cpu_crisper_debug(audio_file, pause_threshold=3.0):
    """
    Debug version to see what's happening
    """
    model_name = 'nyrahealth/faster_CrisperWhisper'
    
    print("Loading CrisperWhisper on CPU...")
    model = WhisperModel(
        model_name, 
        device="cpu",
        compute_type="float32"
    )
    
    print(f"Transcribing: {audio_file}")
    segments, info = model.transcribe(
        audio_file, 
        beam_size=1, 
        language='en', 
        word_timestamps=True,
        vad_filter=True
    )
    
    # Debug: Check if we got any segments
    segments_list = list(segments)  # Convert generator to list
    print(f"DEBUG: Found {len(segments_list)} segments")
    
    if len(segments_list) == 0:
        print("DEBUG: No segments found! Trying without VAD filter...")
        segments, info = model.transcribe(
            audio_file, 
            beam_size=1, 
            language='en', 
            word_timestamps=True,
            vad_filter=False  # Disable voice activity detection
        )
        segments_list = list(segments)
        print(f"DEBUG: After disabling VAD: Found {len(segments_list)} segments")
    
    # Process segments
    transcript = ""
    last_end_time = None
    
    for i, segment in enumerate(segments_list):
        print(f"DEBUG: Processing segment {i+1}: '{segment.text}'")
        
        if last_end_time is not None:
            pause_duration = segment.start - last_end_time
            if pause_duration >= pause_threshold:
                transcript += f" [PAUSE:{pause_duration:.1f}s] "
        
        transcript += segment.text
        last_end_time = segment.end
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
    
    return transcript.strip()

# Run debug version
result = transcribe_with_cpu_crisper_debug("New Recording 48.wav")
print("\nFinal Transcript:")
print("="*50)
print(result)
print("="*50)


Loading CrisperWhisper on CPU...
Transcribing: New Recording 48.wav
DEBUG: Found 0 segments
DEBUG: No segments found! Trying without VAD filter...
DEBUG: After disabling VAD: Found 2 segments
DEBUG: Processing segment 1: 'Hello,mynameisbrandabhani,andiamfromitbranch,Niti,Shibinaur.Iamaveryhappyperson,mylifisverygreat,ihavemanylovelyfriends,andiloveyou.'
[0.00s -> 15.58s] Hello,mynameisbrandabhani,andiamfromitbranch,Niti,Shibinaur.Iamaveryhappyperson,mylifisverygreat,ihavemanylovelyfriends,andiloveyou.
DEBUG: Processing segment 2: 'No,but,I,Ilovemywork,Iamverypassionate,and,currently,Iambuildingaproject,Iambuildingaproject,called,A.I.Interview,in,which.'
[30.40s -> 42.24s] No,but,I,Ilovemywork,Iamverypassionate,and,currently,Iambuildingaproject,Iambuildingaproject,called,A.I.Interview,in,which.

Final Transcript:
Hello,mynameisbrandabhani,andiamfromitbranch,Niti,Shibinaur.Iamaveryhappyperson,mylifisverygreat,ihavemanylovelyfriends,andiloveyou. [PAUSE:14.8s] No,but,I,Ilovemywork,Iamveryp

In [1]:
from faster_whisper import WhisperModel
import re

def transcribe_with_cpu_crisper_clean(audio_file, pause_threshold=2.0):
    """
    Run CrisperWhisper on CPU with cleaned output
    """
    model_name = 'nyrahealth/faster_CrisperWhisper'
    
    print("Loading CrisperWhisper on CPU...")
    model = WhisperModel(
        model_name, 
        device="cpu",
        compute_type="float32"
    )
    
    print(f"Transcribing: {audio_file}")
    segments, info = model.transcribe(
        audio_file, 
        beam_size=1, 
        language='en', 
        word_timestamps=True,
        vad_filter=False  # Keep this disabled since VAD was blocking segments
    )
    
    # Process segments
    transcript = ""
    last_end_time = None
    
    for segment in list(segments):
        if last_end_time is not None:
            pause_duration = segment.start - last_end_time
            if pause_duration >= pause_threshold:
                transcript += f" [PAUSE:{pause_duration:.1f}s] "
        
        # Clean up the segment text
        cleaned_text = clean_transcript_text(segment.text)
        transcript += cleaned_text
        last_end_time = segment.end
        
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {cleaned_text}")
    
    return transcript.strip()

def clean_transcript_text(text):
    """
    Clean up transcript text by removing unwanted commas and fixing spacing
    """
    # Remove commas that separate words (but keep commas that are actual punctuation)
    # Replace commas followed by lowercase letters with spaces
    text = re.sub(r',([a-z])', r' \1', text)
    
    # Replace remaining isolated commas with spaces
    text = re.sub(r',\s*', ' ', text)
    
    # Fix multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Capitalize first letter of sentences
    text = re.sub(r'^\w', lambda m: m.group(0).upper(), text)
    text = re.sub(r'\.\s*\w', lambda m: m.group(0).upper(), text)
    
    # Add proper punctuation spacing
    text = re.sub(r'\.([A-Z])', r'. \1', text)
    
    return text.strip()

# Usage
result = transcribe_with_cpu_crisper_clean("question_1_1757185865994.wav")
print("\nCleaned Final Transcript:")
print("="*60)
print(result)
print("="*60)


c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python313\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading CrisperWhisper on CPU...
Transcribing: question_1_1757185865994.wav
[0.00s -> 10.70s] Hello Hello Hello Matharchod Hello Hello Yehmaphogos Yeh Gir Gir Gir Howareyouall mynameisprayanchod.

Cleaned Final Transcript:
Hello Hello Hello Matharchod Hello Hello Yehmaphogos Yeh Gir Gir Gir Howareyouall mynameisprayanchod.


## YEH Pause short pauses vaala code hai

In [3]:
import librosa
import numpy as np

def detect_audio_pauses(audio_file, pause_threshold=2.0, silence_threshold=-40):
    """
    Detect pauses directly from audio waveform
    """
    # Load audio
    audio, sr = librosa.load(audio_file, sr=16000)
    
    # Convert to decibels
    audio_db = librosa.amplitude_to_db(np.abs(audio), ref=np.max)
    
    # Detect silence (below threshold)
    silence_mask = audio_db < silence_threshold
    
    # Find continuous silence regions
    silence_regions = []
    in_silence = False
    silence_start = 0
    
    for i, is_silent in enumerate(silence_mask):
        if is_silent and not in_silence:
            silence_start = i
            in_silence = True
        elif not is_silent and in_silence:
            silence_duration = (i - silence_start) / sr
            if silence_duration >= pause_threshold:
                silence_regions.append({
                    'start': silence_start / sr,
                    'end': i / sr,
                    'duration': silence_duration
                })
            in_silence = False
    
    print(f"Found {len(silence_regions)} pauses >= {pause_threshold}s:")
    for region in silence_regions:
        print(f"  Pause: {region['start']:.2f}s - {region['end']:.2f}s ({region['duration']:.1f}s)")
    
    return silence_regions

def transcribe_with_audio_pause_detection(audio_file, pause_threshold=2.0):
    """
    Combine CrisperWhisper with audio-based pause detection
    """
    # Get transcription
    model = WhisperModel('nyrahealth/faster_CrisperWhisper', device="cpu", compute_type="float32")
    segments, info = model.transcribe(audio_file, beam_size=1, language='en', vad_filter=False)
    segments_list = list(segments)
    
    # Get audio-based pauses
    audio_pauses = detect_audio_pauses(audio_file, pause_threshold)
    
    # Build transcript with both segment-based and audio-based pauses
    transcript = ""
    
    for i, segment in enumerate(segments_list):
        cleaned_text = clean_transcript_text(segment.text)
        
        # Check for audio-based pauses within this segment
        segment_start = segment.start
        segment_end = segment.end
        
        # Add pauses that occur after this segment
        for pause in audio_pauses:
            if segment_end <= pause['start'] < (segments_list[i+1].start if i+1 < len(segments_list) else float('inf')):
                transcript += f" [PAUSE:{pause['duration']:.1f}s] "
        
        transcript += cleaned_text + " "
        
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {cleaned_text}")
    
    return transcript.strip()

# Usage - install librosa first: pip install librosa
result = transcribe_with_audio_pause_detection("New Recording 48.wav", pause_threshold=2.0)
print("\nTranscript with audio-based pause detection:")
print("="*60)
print(result)
print("="*60)


ModuleNotFoundError: No module named 'librosa'

In [1]:
import librosa
import numpy as np

def detect_audio_pauses(audio_file, pause_threshold=2.0, silence_threshold=-40):
    """
    Detect pauses directly from audio waveform
    """
    # Load audio
    audio, sr = librosa.load(audio_file, sr=16000)
    
    # Convert to decibels
    audio_db = librosa.amplitude_to_db(np.abs(audio), ref=np.max)
    
    # Detect silence (below threshold)
    silence_mask = audio_db < silence_threshold
    
    # Find continuous silence regions
    silence_regions = []
    in_silence = False
    silence_start = 0
    
    for i, is_silent in enumerate(silence_mask):
        if is_silent and not in_silence:
            silence_start = i
            in_silence = True
        elif not is_silent and in_silence:
            silence_duration = (i - silence_start) / sr
            if silence_duration >= pause_threshold:
                silence_regions.append({
                    'start': silence_start / sr,
                    'end': i / sr,
                    'duration': silence_duration
                })
            in_silence = False
    
    print(f"Found {len(silence_regions)} pauses >= {pause_threshold}s:")
    for region in silence_regions:
        print(f"  Pause: {region['start']:.2f}s - {region['end']:.2f}s ({region['duration']:.1f}s)")
    
    return silence_regions

def transcribe_with_audio_pause_detection(audio_file, pause_threshold=2.0):
    """
    Combine CrisperWhisper with audio-based pause detection
    """
    # Get transcription
    model = WhisperModel('nyrahealth/faster_CrisperWhisper', device="cpu", compute_type="float32")
    segments, info = model.transcribe(audio_file, beam_size=1, language='en', vad_filter=False)
    segments_list = list(segments)
    
    # Get audio-based pauses
    audio_pauses = detect_audio_pauses(audio_file, pause_threshold)
    
    # Build transcript with both segment-based and audio-based pauses
    transcript = ""
    
    for i, segment in enumerate(segments_list):
        cleaned_text = clean_transcript_text(segment.text)
        
        # Check for audio-based pauses within this segment
        segment_start = segment.start
        segment_end = segment.end
        
        # Add pauses that occur after this segment
        for pause in audio_pauses:
            if segment_end <= pause['start'] < (segments_list[i+1].start if i+1 < len(segments_list) else float('inf')):
                transcript += f" [PAUSE:{pause['duration']:.1f}s] "
        
        transcript += cleaned_text + " "
        
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {cleaned_text}")
    
    return transcript.strip()

# Usage - install librosa first: pip install librosa
result = transcribe_with_audio_pause_detection("New Recording 48.wav", pause_threshold=2.0)
print("\nTranscript with audio-based pause detection:")
print("="*60)
print(result)
print("="*60)


ModuleNotFoundError: No module named 'librosa'

In [8]:
from faster_whisper import WhisperModel
from pydub import AudioSegment, silence
import re

def transcribe_with_audio_pause_detection(audio_file, pause_threshold=2.0):
    """
    Combine CrisperWhisper transcription with PyDub silence detection
    """
    # Get transcription from CrisperWhisper
    model = WhisperModel('nyrahealth/faster_CrisperWhisper', device="cpu", compute_type="float32")
    segments, info = model.transcribe(
        audio_file, 
        beam_size=1, 
        language='en', 
        word_timestamps=True,
        vad_filter=False
    )
    
    segments_list = list(segments)
    print(f"ASR Segments: {len(segments_list)}")
    
    # Load audio and detect actual silences
    audio = AudioSegment.from_wav(audio_file)
    
    # Detect silences >= pause_threshold seconds
    silence_thresh = -40  # dB - adjust if needed
    silences = silence.detect_silence(
        audio,
        min_silence_len=int(pause_threshold * 1000),  # Convert to milliseconds
        silence_thresh=silence_thresh
    )
    
    print(f"Audio-based silences found: {len(silences)}")
    for i, (start_ms, end_ms) in enumerate(silences):
        duration = (end_ms - start_ms) / 1000
        print(f"  Silence {i+1}: {start_ms/1000:.2f}s - {end_ms/1000:.2f}s ({duration:.1f}s)")
    
    # Build transcript by inserting pauses from audio analysis
    transcript = ""
    
    for i, segment in enumerate(segments_list):
        # Clean segment text
        cleaned_text = clean_transcript_text(segment.text)
        transcript += cleaned_text
        
        # Check if there's a silence after this segment ends
        segment_end_s = segment.end
        
        # Find silences that start near this segment's end
        for silence_start_ms, silence_end_ms in silences:
            silence_start_s = silence_start_ms / 1000
            silence_duration = (silence_end_ms - silence_start_ms) / 1000
            
            # If silence starts within 1 second of segment end, it's likely related
            if abs(silence_start_s - segment_end_s) <= 1.0:
                transcript += f" [PAUSE:{silence_duration:.1f}s] "
                print(f"Added pause after '{cleaned_text}': {silence_duration:.1f}s")
                break
        
        transcript += " "
    
    return transcript.strip()

def clean_transcript_text(text):
    """Clean transcript text"""
    text = re.sub(r',([a-z])', r' \1', text)
    text = re.sub(r',\s*', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    if text:
        text = text[0].upper() + text[1:]
    return text

# Usage
result = transcribe_with_audio_pause_detection("New Recording 49.wav", pause_threshold=2.0)
print("\nTranscript with Audio-Based Pause Detection:")
print("="*60)
print(result)
print("="*60)


ASR Segments: 3
Audio-based silences found: 2
  Silence 1: 22.74s - 26.08s (3.3s)
  Silence 2: 39.61s - 42.50s (2.9s)

Transcript with Audio-Based Pause Detection:
Hello mynameismuthubani and[UH] asyouknow I'mtryingtobuildaproject.Youknow I'mfacingverymuchdifficultiesrightnow because [UH] inmymind there'salotofthingshappeningrightnow [UH]but Ithink so Icanovercomethat because Godisnmyside Iknow I'mjustmakingthisaudio to [UH]tryandtestmy. [UH] I'm. Don't.


In [17]:
from faster_whisper import WhisperModel
from pydub import AudioSegment, silence
import re

def transcribe_with_all_pauses_integrated(audio_file, pause_threshold=2.0):
    """
    Properly integrate ALL detected silences into the transcript
    """
    # Get transcription from CrisperWhisper
    model = WhisperModel('nyrahealth/faster_CrisperWhisper', device="cpu", compute_type="float32")
    segments, info = model.transcribe(
        audio_file, 
        beam_size=1, 
        language='en', 
        word_timestamps=True,
        vad_filter=False
    )
    
    segments_list = list(segments)
    print(f"ASR Segments: {len(segments_list)}")
    
    # Load audio and detect actual silences
    audio = AudioSegment.from_wav(audio_file)
    silences = silence.detect_silence(
        audio,
        min_silence_len=int(pause_threshold * 1000),
        silence_thresh=-40
    )
    
    print(f"Audio-based silences found: {len(silences)}")
    for i, (start_ms, end_ms) in enumerate(silences):
        duration = (end_ms - start_ms) / 1000
        print(f"  Silence {i+1}: {start_ms/1000:.2f}s - {end_ms/1000:.2f}s ({duration:.1f}s)")
    
    # Create timeline events (segments + silences)
    events = []
    
    # Add segment events
    for segment in segments_list:
        events.append({
            'type': 'segment',
            'time': segment.start,
            'end_time': segment.end,
            'text': clean_transcript_text(segment.text)
        })
    
    # Add silence events
    for start_ms, end_ms in silences:
        start_s = start_ms / 1000
        duration = (end_ms - start_ms) / 1000
        events.append({
            'type': 'silence',
            'time': start_s,
            'duration': duration
        })
    
    # Sort all events by time
    events.sort(key=lambda x: x['time'])
    
    # Build transcript chronologically
    transcript = ""
    
    for event in events:
        if event['type'] == 'segment':
            transcript += event['text'] + " "
            print(f"Added text: '{event['text']}'")
        elif event['type'] == 'silence':
            transcript += f"[PAUSE:{event['duration']:.1f}s] "
            print(f"Added pause: {event['duration']:.1f}s at {event['time']:.2f}s")
    
    return transcript.strip()

def clean_transcript_text(text):
    """Clean transcript text"""
    text = re.sub(r',([a-z])', r' \1', text)
    text = re.sub(r',\s*', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    if text:
        text = text[0].upper() + text[1:]
    return text

# Usage
result = transcribe_with_all_pauses_integrated("converted.wav", pause_threshold=2.0)
print("\nTranscript with ALL Pauses Integrated:")
print("="*60)
print(result)
print("="*60)


ASR Segments: 5
Audio-based silences found: 6
  Silence 1: 3.70s - 8.73s (5.0s)
  Silence 2: 12.05s - 15.16s (3.1s)
  Silence 3: 16.16s - 18.82s (2.7s)
  Silence 4: 22.61s - 26.17s (3.6s)
  Silence 5: 28.75s - 32.36s (3.6s)
  Silence 6: 38.44s - 41.34s (2.9s)
Added text: 'Hello friends it'sbeenmyfourthday andnow I'vestartedworkingagain onthesameproject.Still theproblems arenotsolvingthatmuch.Finally I'mabletogetmyfile'sdone.'
Added pause: 5.0s at 3.70s
Added pause: 3.1s at 12.05s
Added pause: 2.7s at 16.16s
Added pause: 3.6s at 22.61s
Added text: '[UM] but still the fuck message now it's still fine.'
Added pause: 3.6s at 28.75s
Added text: 'That's also better.'
Added text: 'Good.'
Added pause: 2.9s at 38.44s
Added text: 'Good.'

Transcript with ALL Pauses Integrated:
Hello friends it'sbeenmyfourthday andnow I'vestartedworkingagain onthesameproject.Still theproblems arenotsolvingthatmuch.Finally I'mabletogetmyfile'sdone. [PAUSE:5.0s] [PAUSE:3.1s] [PAUSE:2.7s] [PAUSE:3.6s] [UM] but still

In [22]:
from faster_whisper import WhisperModel
from pydub import AudioSegment, silence
import re

def transcribe_with_all_pauses_integrated(audio_file, pause_threshold=2.0):
    """
    Properly integrate ALL detected silences into the transcript
    """
    # Get transcription from CrisperWhisper
    model = WhisperModel('nyrahealth/faster_CrisperWhisper', device="cpu", compute_type="float32")
    segments, info = model.transcribe(
        audio_file, 
        beam_size=1, 
        language='en', 
        word_timestamps=True,
        vad_filter=False
    )
    
    segments_list = list(segments)
 
    
    # Load audio and detect actual silences
    audio = AudioSegment.from_wav(audio_file)
    silences = silence.detect_silence(
        audio,
        min_silence_len=int(pause_threshold * 1000),
        silence_thresh=-40
    )
    
   
    for i, (start_ms, end_ms) in enumerate(silences):
        duration = (end_ms - start_ms) / 1000
   
    
    # Create timeline events (segments + silences)
    events = []
    
    # Add segment events
    for segment in segments_list:
        events.append({
            'type': 'segment',
            'time': segment.start,
            'end_time': segment.end,
            'text': clean_transcript_text(segment.text)
        })
    
    # Add silence events
    for start_ms, end_ms in silences:
        start_s = start_ms / 1000
        duration = (end_ms - start_ms) / 1000
        events.append({
            'type': 'silence',
            'time': start_s,
            'duration': duration
        })
    
    # Sort all events by time
    events.sort(key=lambda x: x['time'])
    
    # Build transcript chronologically
    transcript = ""
    
    for event in events:
        if event['type'] == 'segment':
            transcript += event['text'] + " "
           
        elif event['type'] == 'silence':
            transcript += f"[PAUSE:{event['duration']:.1f}s] "
          
    
    return transcript.strip()

def clean_transcript_text(text):
    """Clean transcript text"""
    text = re.sub(r',([a-z])', r' \1', text)
    text = re.sub(r',\s*', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    if text:
        text = text[0].upper() + text[1:]
    return text

# Usage
result = transcribe_with_all_pauses_integrated("converted.wav", pause_threshold=2.0)
print("\nTranscript with ALL Pauses Integrated:")
print("="*60)
print(result)
print("="*60)


KeyboardInterrupt: 

## format sahi kardu

In [6]:
import subprocess
import tempfile
from pathlib import Path 

def convert_to_wav(input_file, output_file=None):
    """Convert audio file to WAV format using ffmpeg"""
    if output_file is None:
        output_file = input_file.replace(Path(input_file).suffix, '_converted.wav')
    
    try:
        # Use ffmpeg to convert the file
        subprocess.run([
            'ffmpeg', '-i', input_file, 
            '-acodec', 'pcm_s16le', 
            '-ar', '16000', 
            '-ac', '1',  # mono
            '-y',  # overwrite output file
            output_file
        ], check=True, capture_output=True)
        
        print(f"Successfully converted to: {output_file}")
        return output_file
    except subprocess.CalledProcessError as e:
        print(f"ffmpeg conversion failed: {e}")
        return None
    except FileNotFoundError:
        print("ffmpeg not found. Please install ffmpeg.")
        return None


In [7]:
convert_to_wav("question_1_1757186277869.wav")

Successfully converted to: question_1_1757186277869_converted.wav


'question_1_1757186277869_converted.wav'

## Webm to WAV file

In [13]:
import subprocess
import os

def convert_webm_to_wav_subprocess(input_file, output_file="output.wav"):
    """Convert WebM to WAV using subprocess"""
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")
    
    try:
        subprocess.run([
            "ffmpeg", "-i", input_file,
            "-acodec", "pcm_s16le",
            "-ar", "16000", 
            "-ac", "1",
            "-y",  # overwrite output
            output_file
        ], check=True, capture_output=True)
        
        print(f"Successfully converted {input_file} to {output_file}")
        return output_file
        
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg error: {e}")
        raise
    except FileNotFoundError:
        print("FFmpeg not found. Please install FFmpeg binary.")
        raise

# Usage
convert_webm_to_wav_subprocess("question_0_1757221339361.webm", "converted.wav")


Successfully converted question_0_1757221339361.webm to converted.wav


'converted.wav'

In [12]:
import ffmpeg
print(dir(ffmpeg))  # Should show 'input' in the list


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [11]:
import ffmpeg

(
    ffmpeg
    .input("question_0_1757221339361.webm")
    .output("output.wav", acodec="pcm_s16le", ar=16000, ac=1)
    .run()
)


AttributeError: module 'ffmpeg' has no attribute 'input'

## Audio analyze

In [2]:
from faster_whisper import WhisperModel
from pydub import AudioSegment, silence
import re
import os

def fix_audio_for_transcription(audio_file):
    """
    Fix the specific issues found in your audio files
    """
    print(f"Preprocessing {audio_file}...")
    
    # Load audio
    audio = AudioSegment.from_wav(audio_file)
    
    # Fix sample rate (your main issue)
    if audio.frame_rate != 16000:
        print(f"  ✓ Converting {audio.frame_rate}Hz → 16kHz")
        audio = audio.set_frame_rate(16000)
    
    # Fix clipping issues (your second issue)
    if audio.max_dBFS > -6:
        reduction = audio.max_dBFS + 12  # Target -12dBFS
        audio = audio - reduction
        print(f"  ✓ Reduced clipping: -{reduction:.1f}dB")
    
    # Ensure mono
    if audio.channels > 1:
        audio = audio.set_channels(1)
        print("  ✓ Converted to mono")
    
    # Save fixed audio
    base_name = os.path.splitext(audio_file)[0]
    fixed_file = f"{base_name}_fixed.wav"
    audio.export(fixed_file, format="wav")
    print(f"  ✓ Saved: {fixed_file}")
    
    return fixed_file

def transcribe_with_audio_fixes(audio_file, pause_threshold=2.0):
    """
    Your existing transcription function with audio preprocessing
    """
    # Fix audio issues first
    try:
        fixed_file = fix_audio_for_transcription(audio_file)
        use_file = fixed_file
    except Exception as e:
        print(f"Preprocessing failed: {e}, using original file")
        use_file = audio_file
    
    try:
        # Get transcription from CrisperWhisper
        model = WhisperModel('nyrahealth/faster_CrisperWhisper', device="cpu", compute_type="float32")
        segments, info = model.transcribe(
            use_file, 
            beam_size=3,  # Higher beam size for better quality
            language='en', 
            word_timestamps=True,
            vad_filter=False,
            temperature=0.0,  # More deterministic
            condition_on_previous_text=False  # Prevent context bias
        )
        
        segments_list = list(segments)
        print(f"ASR Segments: {len(segments_list)}")
        
        if len(segments_list) == 0:
            print("No segments detected - trying with different parameters...")
            # Try again with more permissive settings
            segments, info = model.transcribe(
                use_file,
                beam_size=1,
                language='en',
                word_timestamps=True,
                vad_filter=False,
                compression_ratio_threshold=1.8,
                logprob_threshold=-1.5
            )
            segments_list = list(segments)
        
        # Load audio and detect actual silences
        audio = AudioSegment.from_wav(use_file)
        silences = silence.detect_silence(
            audio,
            min_silence_len=int(pause_threshold * 1000),
            silence_thresh=-35  # Slightly more sensitive for processed audio
        )
        
        print(f"Audio-based silences found: {len(silences)}")
        for i, (start_ms, end_ms) in enumerate(silences):
            duration = (end_ms - start_ms) / 1000
            print(f"  Silence {i+1}: {start_ms/1000:.2f}s - {end_ms/1000:.2f}s ({duration:.1f}s)")
        
        # Create timeline events (segments + silences)
        events = []
        
        # Add segment events
        for segment in segments_list:
            events.append({
                'type': 'segment',
                'time': segment.start,
                'end_time': segment.end,
                'text': clean_transcript_text(segment.text)
            })
        
        # Add silence events
        for start_ms, end_ms in silences:
            start_s = start_ms / 1000
            duration = (end_ms - start_ms) / 1000
            events.append({
                'type': 'silence',
                'time': start_s,
                'duration': duration
            })
        
        # Sort all events by time
        events.sort(key=lambda x: x['time'])
        
        # Build transcript chronologically
        transcript = ""
        
        for event in events:
            if event['type'] == 'segment':
                transcript += event['text'] + " "
                print(f"Added text: '{event['text']}'")
            elif event['type'] == 'silence':
                transcript += f"[PAUSE:{event['duration']:.1f}s] "
                print(f"Added pause: {event['duration']:.1f}s at {event['time']:.2f}s")
        
        # Cleanup temp file
        if use_file != audio_file and os.path.exists(use_file):
            os.remove(use_file)
        
        return transcript.strip()
        
    except Exception as e:
        print(f"Transcription failed: {e}")
        return "Transcription failed"

def clean_transcript_text(text):
    """Clean transcript text"""
    text = re.sub(r',([a-z])', r' \1', text)
    text = re.sub(r',\s*', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    if text:
        text = text[0].upper() + text[1:]
    return text

# Usage - this should now work consistently
result = transcribe_with_audio_fixes("Recording (6).wav", pause_threshold=2.0)
print("\nFixed Transcript:")
print("="*60)
print(result)
print("="*60)


c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python313\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preprocessing Recording (6).wav...
  ✓ Converting 48000Hz → 16kHz
  ✓ Converted to mono
  ✓ Saved: Recording (6)_fixed.wav
ASR Segments: 0
No segments detected - trying with different parameters...
Transcription failed: WhisperModel.transcribe() got an unexpected keyword argument 'logprob_threshold'. Did you mean 'log_prob_threshold'?

Fixed Transcript:
Transcription failed
